<a href="https://colab.research.google.com/github/Asigen93/deteksiayam_yolov8_and_deepsort/blob/main/beta6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# jalankan di Colab cell (prefix !)
!pip install -U pip
!pip install ultralytics roboflow opencv-python-headless matplotlib tqdm pandas
# TensorFlow dibutuhkan untuk export/validasi TFLite
!pip install -U "tensorflow>=2.11.0"
!pip install -U yolox-bytetrack
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 61.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 100.8 MB/s  0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [ultralytics]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="QIxsCi9aE9bKIfs7vXkg")  # paste API key Anda
# Ganti workspace/project/version sesuai Roboflow Anda:
# Anda bisa dapatkan nama project & versi dari URL Roboflow atau dari dashboard
project = rf.workspace().project("countingayam-om39k-jj99v")  # jika error, lihat catatan di bawah
dataset = project.version(1).download("yolov8")  # pilihan export: "yolov8" atau "yolov5"
print("Dataset downloaded to:", dataset.location)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to countingayam-1 in yolov8:: 100%|██████████| 2726/2726 [00:13<00:00, 206.33it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset downloaded to: /content/countingayam-1


In [3]:
import os
# sesuaikan path kalau Anda pakai dataset.location dari SDK, contohnya:
dataset_path = "/content/countingayam-1"   # ganti sesuai lokasi unzip / dataset.location
# kalau pakai SDK: dataset_path = dataset.location

for root, dirs, files in os.walk(dataset_path):
    print(root, "->", len([f for f in files if f.endswith(('.jpg','.png'))]), "images")
    # break supaya tidak print semua
    break

# Jika Roboflow sudah menyertakan data.yaml, gunakan itu.
!sed -n '1,200p' {dataset_path}/data.yaml

/content/countingayam-1 -> 0 images
names:
- ayam
nc: 1
roboflow:
  license: CC BY 4.0
  project: countingayam-om39k-jj99v
  url: https://universe.roboflow.com/dianrcs/countingayam-om39k-jj99v/dataset/1
  version: 1
  workspace: dianrcs
test: ../test/images
train: ../train/images
val: ../valid/images


In [4]:
print(dataset.location)

/content/countingayam-1


In [5]:
from ultralytics import YOLO

# Load arsitektur YOLOv11 kecil (nano)
model = YOLO("yolov8m.yaml")

# Training dengan dataset ayam
model.train(
    data="countingayam-1/data.yaml",
    epochs=50,
    imgsz=960,          # ukuran input, bisa diubah misalnya 960
    batch=16,           # ukuran batch training
    device=0            # 0 = GPU, 'cpu' jika tanpa GPU
)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=countingayam-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78cb7d340050>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [6]:
# Validasi
results = model.val(data="/content/countingayam-1/data.yaml")
print(results)

NameError: name 'model' is not defined

In [5]:
# Prediksi contoh gambar di val/images
model.predict(
    source="/content/countingayam-1/valid/images",
    save=True,
    conf=0.25,
    imgsz=960
)

NameError: name 'model' is not defined

In [ ]:
from IPython.display import Image, display

# Tampilkan confusion matrix
display(Image(filename="/content/runs/detect/train/confusion_matrix.png"))

In [ ]:
from IPython.display import Image, display

# Tampilkan confusion matrix
display(Image(filename="/content/runs/detect/train/results.png"))

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker

# --- Konfigurasi ByteTrack (sama seperti sebelumnya) ---
class BYTETrackerArgs:
    track_thresh = 0.5
    track_buffer = 30
    match_thresh = 0.8
    aspect_ratio_thresh = 3.0
    min_box_area = 10
    mot20 = False

def infer_and_count_video_with_antennas(
    video_path,
    weight_path,
    labels=['ayam'],
    out_video='out_count_bytetrack_ant.mp4',
    conf_thres=0.25,
    iou_thres=0.45,
    imgsz=960,
    expected_bird_width=120,   # asumsi lebar rata-rata 1 ayam di piksel (sesuaikan)
    max_antennas=3,            # maksimal antena per box
    antenna_tolerance_px=5     # toleransi untuk menganggap antena menyentuh garis
):
    # Load model
    model = YOLO(weight_path)

    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Video tidak ditemukan: {video_path}")
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_video, fourcc, fps, (w, h))

    # Tracker
    tracker = BYTETracker(BYTETrackerArgs(), frame_rate=fps)

    # Counting state
    total_count = 0
    counted_antenna_ids = set()   # (track_id, antenna_idx) yang sudah dihitung
    antenna_history = {}          # track_id -> [x_ant1, x_ant2, ...] dari frame sebelumnya

    # Garis vertical tengah
    line_x = int(w * 0.5)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        # YOLO inference
        results = model.predict(frame, imgsz=imgsz, conf=conf_thres, iou=iou_thres, verbose=False)[0]

        # Build detections array for bytetrack: [x, y, w, h, score, cls]
        detections = []
        for r in results.boxes:
            # r.xyxy is tensor shape (1,4); convert to numpy
            x1, y1, x2, y2 = r.xyxy[0].cpu().numpy()
            conf = float(r.conf.cpu().numpy()[0]) if hasattr(r.conf, 'cpu') else float(r.conf)
            cls = int(r.cls.cpu().numpy()[0]) if hasattr(r.cls, 'cpu') else int(r.cls)
            w_box = x2 - x1
            h_box = y2 - y1
            detections.append([x1, y1, w_box, h_box, conf, cls])

        if len(detections) == 0:
            online_targets = []
        else:
            online_targets = tracker.update(np.array(detections), (h, w), (h, w))

        # process tracks
        for t in online_targets:
            if t is None:
                continue
            tlwh = t.tlwh  # tlwh
            x1 = int(tlwh[0]); y1 = int(tlwh[1])
            bw = int(tlwh[2]); bh = int(tlwh[3])
            x2 = x1 + bw; y2 = y1 + bh
            track_id = t.track_id

            # choose label
            cls_id = int(t.class_id) if hasattr(t, "class_id") else 0
            label = labels[cls_id] if cls_id < len(labels) else str(cls_id)

            # Draw YOLO-style box (use tracker box for stability but you can also draw YOLO's box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(frame, f"{label} ID:{track_id}", (x1, max(y1-10, 0)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 3)

            # Determine number of antennas for this box adaptively
            # If box width is larger than expected_bird_width * 1.5, allow up to max_antennas
            est = max(1, int(round(bw / float(max(1, expected_bird_width)))))
            n_ant = min(max_antennas, max(1, est))
            # choose positions as fractions across width
            if n_ant == 1:
                fracs = [0.5]
            elif n_ant == 2:
                fracs = [0.33, 0.67]
            else:
                fracs = [0.25, 0.5, 0.75]

            # compute antenna x positions and draw small vertical antenna lines
            curr_antennas = []
            for idx, f in enumerate(fracs):
                ax = int(x1 + f * bw)
                curr_antennas.append(ax)
                # draw antenna for visualization (short vertical line inside bbox)
                ay1 = int(y1 + bh * 0.15)
                ay2 = int(y2 - bh * 0.15)
                cv2.line(frame, (ax, ay1), (ax, ay2), (0, 255, 255), 2)
                # small circle on antenna
                cv2.circle(frame, (ax, (ay1+ay2)//2), 3, (0, 255, 255), -1)

            # Compare antenna positions with history to detect crossing
            prev_ant = antenna_history.get(track_id, None)
            if prev_ant is not None:
                # iterate over min length antennas (if number changed, handle indices available)
                for ai in range(min(len(prev_ant), len(curr_antennas))):
                    prev_x = prev_ant[ai]
                    curr_x = curr_antennas[ai]
                    # crossing left->right
                    if prev_x < line_x and curr_x >= line_x and (track_id, ai) not in counted_antenna_ids:
                        counted_antenna_ids.add((track_id, ai))
                        total_count += 1
                    # crossing right->left
                    elif prev_x > line_x and curr_x <= line_x and (track_id, ai) not in counted_antenna_ids:
                        counted_antenna_ids.add((track_id, ai))
                        total_count += 1
                # If new antenna indices appeared (e.g., box expanded), we can initialize their prev position
                # but don't count immediately; will be compared next frame
                if len(curr_antennas) > len(prev_ant):
                    # pad prev with current for new ones so they won't count immediately
                    for ai in range(len(prev_ant), len(curr_antennas)):
                        # set previous equal to current to avoid false crossing
                        prev_ant.append(curr_antennas[ai])
            # store current antenna positions as history for this track
            antenna_history[track_id] = curr_antennas

        # Draw counting line and total
        cv2.line(frame, (line_x, 0), (line_x, h), (255, 0, 0), 3)
        cv2.putText(frame, f"Total count: {total_count}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)

        writer.write(frame)

    cap.release()
    writer.release()
    print("Selesai. Output:", out_video)
    return out_video

# --- contoh panggil ---
# pastikan byte track dan yolov11/yolov8 tersedia dan model path benar
out = infer_and_count_video_with_antennas(
    video_path='/content/tes1.mp4',
    weight_path='runs/detect/train/weights/best.pt',
    labels=['broiler'],        # label model Anda (cek model.names jika perlu)
    out_video='result_count_bytetrack_ant.mp4',
    conf_thres=0.25,
    iou_thres=0.2,
    imgsz=960,
    expected_bird_width=120,
    max_antennas=3
)

In [ ]:
from ultralytics import YOLO

# path weight hasil training:
weight_path = "runs/detect/train/weights/best.pt"
model = YOLO(weight_path)

# 1) Export FP16 (smaller, default no int8):
model.export(format="tflite", imgsz=960, optimize=True)
# Hasil: file seperti best-fp16.tflite di working dir

# 2) (Opsional) Export INT8 (post-training quantization)
# Pastikan dataset/data.yaml ada, Ultralytics akan gunakan data untuk calibrasi quantization
try:
    model.export(format="tflite", imgsz=640, int8=True, data='data.yaml', optimize=True)
    print("INT8 export selesai.")
except Exception as e:
    print("INT8 export gagal, cek error:", e)
    print("Gunakan fallback FP16 atau lakukan quantization manual.")

In [ ]:
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="best-fp16.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input:", input_details)
print("Output:", output_details)

# contoh preprocess satu image:
import cv2, numpy as np
img = cv2.imread('/content/some_test_image.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = input_details[0]['shape'][1], input_details[0]['shape'][2]
img_resized = cv2.resize(img, (w,h))
img_arr = img_resized.astype('float32') / 255.0
img_arr = np.expand_dims(img_arr, axis=0)

interpreter.set_tensor(input_details[0]['index'], img_arr)
interpreter.invoke()
outs = [interpreter.get_tensor(o['index']) for o in output_details]
for o in outs:
    print("out shape:", o.shape)
# inspeksi array untuk tahu format keluaran; parsing custom diperlukan